In [1]:
import pandas as pd
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize,wordpunct_tokenize,sent_tokenize 
from nltk.corpus import stopwords 
import string
import re

In [2]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC, NuSVC,SVC
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
from sklearn.metrics import confusion_matrix, f1_score,precision_score,recall_score, classification_report
from sklearn.metrics import precision_recall_fscore_support

In [3]:
#read the data and define the columns
df1 = pd.read_csv('./data-2_train.csv')
df1.columns = ['example_id', 'text', 'aspect_term', 'term_location', 'class']



In [4]:
#testing n gram

In [5]:
data = df1.copy()
data2 = data.copy()
vec = TfidfVectorizer(min_df = 0.00125, max_df = 0.7, sublinear_tf=True, use_idf=True, stop_words=u'english', analyzer= 'word', ngram_range=(1,5),lowercase=True)
X = vec.fit_transform(data['text'])
svm = LinearSVC(C=1.2)
pred_weights = cross_val_predict(svm,X,data['class'],cv = 10)
np.mean(pred_weights == data['class'])

0.6410327595780122

In [6]:
#intitalize count vectorizer
countVectorizer = CountVectorizer()
#get the list of vocab
count_matrix = countVectorizer.fit_transform(df1['text'])
vocab = list(countVectorizer.vocabulary_.keys())
len(vocab)

3716

In [7]:
df1.head()

,example_id,text,aspect_term,term_location,class
0,3121_0,But the staff was so horrible to us.,staff,8--13,-1
1,2777_0,To be completely fair[comma] the only redeemin...,food,57--61,1
2,1634_0,The food is uniformly exceptional[comma] with ...,food,4--8,1
3,1634_1,The food is uniformly exceptional[comma] with ...,kitchen,55--62,1
4,1634_2,The food is uniformly exceptional[comma] with ...,menu,141--145,0


In [8]:
def processData(data):
    #replace [comma] with a ','
    data = data.replace("[comma]",",")
    
    #tokenize with punctuations
    data = " ".join(wordpunct_tokenize(data))
    
    #remove punctuations
    nopunc = [char for char in data if char not in string.punctuation]
    
    #join the words and remove stopswords
    data = "".join(nopunc)
    data = [text for text in data.strip().split() if text not in set(stopwords.words('english'))]  
    #message = [snowball.stem(w) for w in message]
    
    #convert the text into word tokens
    data = " ".join(data)
    words = word_tokenize(data)
    
    #lemmatize using wordNetLemmatizer
    from nltk.stem import WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()
    data = [lemmatizer.lemmatize(word) for word in words]
    
    #return processed data
    return data



In [9]:
#process text and aspect term columns of input data
df1['clean_text'] = df1['text'].apply(processData)
df1['clean_aspect_term'] = df1['aspect_term'].apply(processData)

In [10]:
df1.head()

,example_id,text,aspect_term,term_location,class,clean_text,clean_aspect_term
0,3121_0,But the staff was so horrible to us.,staff,8--13,-1,"[But, staff, horrible, u]",[staff]
1,2777_0,To be completely fair[comma] the only redeemin...,food,57--61,1,"[To, completely, fair, redeeming, factor, food...",[food]
2,1634_0,The food is uniformly exceptional[comma] with ...,food,4--8,1,"[The, food, uniformly, exceptional, capable, k...",[food]
3,1634_1,The food is uniformly exceptional[comma] with ...,kitchen,55--62,1,"[The, food, uniformly, exceptional, capable, k...",[kitchen]
4,1634_2,The food is uniformly exceptional[comma] with ...,menu,141--145,0,"[The, food, uniformly, exceptional, capable, k...",[menu]


In [11]:
#method to assign weights
def assign_weights(data):
    
    #get the te
    text = data[0]
    aspect =data[1]
    
    #corner case to check aspect terms are lesser or equal to the text
    if set(aspect) <= set(text):
        
        #to store left and right weights
        leftWeights = rightWeights = []
        
        #find the starting point of the first aspect term word
        aspect_start = [i for i, word in enumerate(text) if word == aspect[0]]
        
        #find the start and end indices of the aspect
        for pos in (aspect_start):
            if  text[(pos + len(aspect) - 1)] == aspect[-1]:
                start_index = pos
                end_index = pos + len(aspect) - 1
                break
                
        #assign weights to each word moving left and right of the aspect term         
        if (end_index - start_index) == len(aspect) - 1:
            #get the left side and the right side of the text from the aspect
            left_text = text[:start_index]
            right_text = text[end_index+1:]
            
            #generate left and right weights based on a well known strategy
            leftWeights = [1/i for i in range(len(left_text),0,-1) if len(left_text) != 0]
            rightWeights = [1/i for i in range(1,len(right_text)+1) if len(right_text) != 0]
         
        #find the total weights by adding a constant
        tot_weights = leftWeights + [2]*len(aspect) + rightWeights
        
        #return the dict of text and its weights
        return dict(zip(text,tot_weights))
    else: 
        return np.nan


In [27]:
#assign weights to the data
df1['element_weights'] = df1[['clean_text','clean_aspect_term']].apply(assign_weights, axis = 1)

#drop all nan values
df1 = df1.dropna()

#create an empty df with zeros
df2 = pd.DataFrame(np.zeros((len(df1),len(vocab))),columns=vocab)

df1.head()

,example_id,text,aspect_term,term_location,class,clean_text,clean_aspect_term,weights_score,element_weights
0,3121_0,But the staff was so horrible to us.,staff,8--13,-1,"[But, staff, horrible, u]",[staff],"{'But': 1.0, 'staff': 2, 'horrible': 1.0, 'u':...","{'But': 1.0, 'staff': 2, 'horrible': 1.0, 'u':..."
1,2777_0,To be completely fair[comma] the only redeemin...,food,57--61,1,"[To, completely, fair, redeeming, factor, food...",[food],"{'To': 0.2, 'completely': 0.25, 'fair': 0.3333...","{'To': 0.2, 'completely': 0.25, 'fair': 0.3333..."
2,1634_0,The food is uniformly exceptional[comma] with ...,food,4--8,1,"[The, food, uniformly, exceptional, capable, k...",[food],"{'The': 1.0, 'food': 2, 'uniformly': 1.0, 'exc...","{'The': 1.0, 'food': 2, 'uniformly': 1.0, 'exc..."
3,1634_1,The food is uniformly exceptional[comma] with ...,kitchen,55--62,1,"[The, food, uniformly, exceptional, capable, k...",[kitchen],"{'The': 0.2, 'food': 0.25, 'uniformly': 0.3333...","{'The': 0.2, 'food': 0.25, 'uniformly': 0.3333..."
4,1634_2,The food is uniformly exceptional[comma] with ...,menu,141--145,0,"[The, food, uniformly, exceptional, capable, k...",[menu],"{'The': 0.07692307692307693, 'food': 0.0833333...","{'The': 0.07692307692307693, 'food': 0.0833333..."


In [28]:
# assign the calculated weights to the corresponding text elements of df2
for row in range(len(df1)):
    for key,value in df1.iloc[row]['element_weights'].items():
        df2.iloc[row][key] = value
        
#feature extraction - transform a count matrix to a normalized tf-idf representation
tfidf= TfidfTransformer().fit_transform(df2)


In [14]:
## SVM 

In [15]:
svm = LinearSVC(C=1.2)
pred_SVM = cross_val_predict(svm,tfidf,df1['class'],cv = 10)

np.mean(pred_SVM == df1['class'])

0.6613888888888889

In [16]:
svm.fit(tfidf, df1['class'])

LinearSVC(C=1.2, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [19]:
print(precision_recall_fscore_support(df1['class'], pred_SVM, labels=[-1,0,1]))

print("\n Classification Report \n ", classification_report(pred_SVM,df1['class']))

(array([0.55610561, 0.42950108, 0.7287801 ]), array([0.41915423, 0.31279621, 0.85344429]), array([0.47801418, 0.36197441, 0.78620102]), array([ 804,  633, 2163], dtype=int64))

 Classification Report 
                precision    recall  f1-score   support

          -1       0.42      0.56      0.48       606
           0       0.31      0.43      0.36       461
           1       0.85      0.73      0.79      2533

   micro avg       0.66      0.66      0.66      3600
   macro avg       0.53      0.57      0.54      3600
weighted avg       0.71      0.66      0.68      3600



In [ ]:
# Decision tree classifier

In [ ]:
vec = TfidfVectorizer(min_df = 0.00125, max_df = 0.7, sublinear_tf=True, use_idf=True, stop_words=u'english', analyzer= processData, ngram_range=(1,5),lowercase=True)

In [ ]:
X = vec.fit_transform(df1['text'])

In [ ]:
Y = df1['class']

In [ ]:
DT = DecisionTreeClassifier(max_depth=10).fit(X,Y)

In [ ]:
pred = cross_val_predict(DT,X,Y,cv=10)

In [ ]:
np.mean(Y==pred)

In [ ]:
print(precision_recall_fscore_support(Y, pred, labels=[-1,0,1]))

print("\n Classification Report \n ", classification_report(pred,Y))

In [ ]:
# Multinomial NB

In [ ]:
clf_NB = MultinomialNB().fit(X,Y)

In [ ]:
pred_NB= cross_val_predict(clf_NB,X,Y,cv=10)

In [ ]:
np.mean(Y==pred_NB)

In [ ]:
print(precision_recall_fscore_support(Y, pred_NB, labels=[-1,0,1]))

print("\n Classification Report \n ", classification_report(pred_NB,Y))

In [ ]:
# Random forest

In [34]:
clf_RFC = RandomForestClassifier(n_estimators=100,max_depth=200)
pred_RF = cross_val_predict(clf_RFC,df2,df1['class'],cv = 10)
np.mean(pred_RF == df1['class'])

0.6566666666666666

In [37]:
print(precision_recall_fscore_support(df1['class'], pred_RF, labels=[-1,0,1]))

print("\n Classification Report \n ", classification_report(pred_RF,df1['class']))

(array([0.62678063, 0.44015444, 0.67892977]), array([0.27363184, 0.18009479, 0.93851133]), array([0.38095238, 0.25560538, 0.78789055]), array([ 804,  633, 2163], dtype=int64))

 Classification Report 
                precision    recall  f1-score   support

          -1       0.27      0.63      0.38       351
           0       0.18      0.44      0.26       259
           1       0.94      0.68      0.79      2990

   micro avg       0.66      0.66      0.66      3600
   macro avg       0.46      0.58      0.47      3600
weighted avg       0.82      0.66      0.71      3600



In [ ]:
# Testing

In [31]:
#define the columns required
testcols=['example_id','text','aspect_term','term_location']
df_test = pd.read_csv('./Data-2_test.csv',skipinitialspace=True, usecols=testcols)

df_test.columns = ['example_id','text','aspect_term','term_location']
countVectorizer = CountVectorizer()
count_matrix = countVectorizer.fit_transform(df_test['text'])
df_test['text'] = df_test['text'].apply(processData)
df_test['processed_AT'] = df_test['aspect_term'].apply(processData)
df_test['element_weights'] = df_test[['text','processed_AT']].apply(assign_weights, axis = 1)
df_test = df_test.dropna()
df2_test = pd.DataFrame(np.zeros((len(df1),len(vocab))),columns=vocab)

#assign weights and add it to the dict
for row in range(len(df_test)):    
    for key,value in df_test.iloc[row]['element_weights'].items():
        df2_test.iloc[row][key] = value
tfidf= TfidfTransformer().fit_transform(df2_test)

#do a single pass over the test data using existing classifier 
pred_SVM = svm.predict(df2_test)
res = list()
file = open('KruneetKumar_Patel_Vishwas_SreevalliRamamohan_Data-2.txt','w')

#zip example ids with their predictions and write it to a file
for x,y in zip(list(df_test['example_id']),pred_SVM):
    res.append(str(x) +";;"+str(y))
for i in res:
    file.write("%s\n"%i)
file.close()

In [33]:
i = 0 
with open('Data-2.txt','r') as f:
    x = f.readlines()
    for y in x:
        i += 1 
print(i)


1120


In [24]:
np.mean(pred_SVM==[-1,0,1])

1120

In [ ]:
# Experimenting

In [ ]:
data2 = df1.copy()
df1['clean_text']= " ".join(df1['clean_text'])
vec = TfidfVectorizer(min_df = 0.00125, max_df = 0.7, sublinear_tf=True, use_idf=True, stop_words=u'english', analyzer= 'word', ngram_range=(1,5),lowercase=True)
X = vec.fit_transform(data['text'])
svm = LinearSVC(C=1.2)
pred_weights = cross_val_predict(svm,X,data['class'],cv = 10)
np.mean(pred_weights == data['class'])

In [ ]:
df1['sentText'] = [" ".join(x) for x in df1['clean_text']]

In [ ]:
df1['sentAspect'] = [" ".join(c) for c in df1['clean_aspect_term']]

In [ ]:
vec = TfidfVectorizer(min_df = 0.00125, max_df = 0.5, sublinear_tf=True, use_idf=True, stop_words=u'english', analyzer= 'word', ngram_range=(1,10),lowercase=True)

In [ ]:
X = vec.fit_transform(df1['sentText'])

In [ ]:
len(df1['class'])

In [ ]:
svm = LinearSVC(C=1.2)
pred_weights = cross_val_predict(svm, X ,df1['class'],cv = 10)
len(pred_weights)

In [ ]:
np.mean(pred_weights == df1['class'])